In [1]:
# OpenTripMap API requests

import json
from dotenv import load_dotenv
load_dotenv()

import time
import os

OpenTrip_API = os.environ.get('OpenTripMap_API_key')

import requests

Base_url = 'https://api.opentripmap.com/0.1/en/places/radius'

params = {
    'radius': 1000,
    'lon': 121.5170,
    'lat': 25.0478,
    'apikey': OpenTrip_API,
    'format': 'json'
}

response = requests.get(Base_url, params = params)

print(response.status_code)
print(response.text)

200
{}


In [3]:
# TDX API requests


import requests
from pprint import pprint
import time
import json

from dotenv import load_dotenv
load_dotenv()

import os

TDX_API_ID = os.environ.get('TDX_APP_ID')
TDX_API_KEY = os.environ.get('TDX_API_KEY')

app_id = TDX_API_ID
app_key = TDX_API_KEY

auth_url = 'https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token'
url = 'https://tdx.transportdata.tw/api/tourism/service/odata/V2/Tourism/Nearby?X=121.5170&Y=25.0478&Distance=1000&%24select=RelatedAttractions'

class Auth():
    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        content_type = 'application/x-www-form-urlencoded'
        grant_type = 'client_credentials'

        return{
            'content_type': content_type,
            'grant_type': grant_type,
            'client_id': self.app_id,
            'client_secret': self.app_key
        }

class data():
    def __init__(self, app_id, app_key, auth_response):
        self.app_id = app_id
        self.app_key = app_key
        self.auth_response = auth_response

    def get_data_header(self):
        auth_JSON = json.loads(self.auth_response.text)
        access_token = auth_JSON.get('access_token')

        return{
            'authorization' : 'Bearer ' + access_token,
            'accept-Encoding' : 'gzip'
        }

if __name__ == '__main__':
    try:
        d = data(app_id, app_key, auth_response)
        data_response = requests.get(url, headers = d.get_data_header())
    except:
        a = Auth(app_id, app_key)
        auth_response = requests.post(auth_url, a.get_auth_header())
        d = data(app_id, app_key, auth_response)
        data_response = requests.get(url, headers = d.get_data_header())
    print(auth_response)
    pprint(auth_response.text)
    print(data_response)
    pprint(data_response.text)



with open('Taipei_station_attractions.json', 'w', encoding = 'utf-8') as f:
    json_data = json.loads(data_response.text)
    # json.dumps將程式作用的對象(像是字串、列表)進行格式化輸出
    formatted_data = json.dumps(json_data, indent = 4, sort_keys = True, ensure_ascii = False)
    f.write(formatted_data)

print(formatted_data)

with open('Taipei_station_attractions.json', 'r', encoding = 'utf-8') as f1:
    attractions_dict = json.load(f1)

print(type(attractions_dict))
print(attractions_dict)

attraction_list = attractions_dict['RelatedAttractions']
print(type(attraction_list))
print(attraction_list)

source_location = {
    'name': '台北車站',
    'Lat': 25.0478,
    'Lon': 121.5170
}

target_locations = attraction_list

from geopy.distance import geodesic

def get_distances(source_location, target_locations):
    results = [
        {
            'AttractionName': target_location['AttractionName'],
            'distance(km)': geodesic((source_location['Lat'], source_location['Lon']), (target_location['PositionLat'], target_location['PositionLon'])).kilometers
        }
        for target_location in target_locations
    ]
    return results

results = get_distances(source_location, target_locations)
print(results)

import pandas as pd

# 原始資料轉成DataFrame
df = pd.DataFrame(attraction_list)

# 計算距台北車站距離的結果資料轉成DataFrame
df1 = pd.DataFrame(results)

# 依據'AttractionName'欄位，合併以上2個DataFrame
final = pd.merge(df, df1, on = 'AttractionName')
final

# final.to_csv('attraction_distance_to_Taipei_station.csv', index = 0, header = 1, encoding = 'utf-8')

<Response [200]>
('{"access_token":"eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJER2lKNFE5bFg4WldFajlNNEE2amFVNm9JOGJVQ3RYWGV6OFdZVzh3ZkhrIn0.eyJleHAiOjE3NjMzOTE4NDEsImlhdCI6MTc2MzMwNTQ0MSwianRpIjoiNDNkNDBkYjQtZDgxYS00NDc4LTlmODEtNTAxMTU1YTAzODRlIiwiaXNzIjoiaHR0cHM6Ly90ZHgudHJhbnNwb3J0ZGF0YS50dy9hdXRoL3JlYWxtcy9URFhDb25uZWN0Iiwic3ViIjoiYjg0ZWZjOGMtM2JlOS00ZjgzLWExMzUtNDQ3YzYxYWIzNWZiIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiY3dlbmhzdWFuLTdmZDhjNjJhLTIwNzItNGIxNiIsImFjciI6IjEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsic3RhdGlzdGljIiwicHJlbWl1bSIsInBhcmtpbmdGZWUiLCJtYWFzIiwiYWR2YW5jZWQiLCJnZW9pbmZvIiwidmFsaWRhdG9yIiwidG91cmlzbSIsImhpc3RvcmljYWwiLCJjd2EiLCJiYXNpYyJdfSwic2NvcGUiOiJwcm9maWxlIGVtYWlsIiwidXNlciI6IjI3MjVhYWMwIn0.MYW-cSzbPXdeeLZYjyqK8EqW_HkOaXlBpkddbtiF5glKdelLF0_CxIc9XTa8OGjyIGuD6sUsdlePPxTc4uyh3-UEJwcSTb7Ub4nHK8jXAEzatlMMm6bNmdItRFM8XWhUVlg6j4T8mN7IvjWGyzkSTYiTytulkXLmPCIA4Xo5uDP91WQ0fPBMTUW-27jsZy_05xTj3GtY6A2YiVsDiRp-rhoIyYW_XkB2lCGWq32aX-lhi02USV4uKyo4DQWUKQ1A0hgMsgFLaFDpAuMZPyKqOChg

,AttractionID,AttractionName,PositionLat,PositionLon,distance(km)
0,Attraction_379000000A_000104,國立臺灣博物館,25.042760,121.515030,0.592637
1,Attraction_379000000A_000137,二二八和平公園,25.041320,121.515160,0.741439
2,Attraction_379000000A_000141,台北當代藝術館,25.050720,121.518920,0.377046
3,Attraction_379000000A_000158,SPOT光點台北電影館,25.053250,121.522250,0.803200
4,Attraction_379000000A_000368,北門_承恩門,25.047730,121.511230,0.582308
5,Attraction_379000000A_000385,國立臺灣大學醫學院附設醫院_西址,25.041990,121.517220,0.643977
6,Attraction_379000000A_000387,東門_景福門,25.039070,121.517650,0.969275
7,Attraction_379000000A_000389,台灣基督長老教會_中山教會,25.049030,121.524380,0.757081
8,Attraction_379000000A_000394,臺北賓館,25.040150,121.516750,0.847793
9,Attraction_379000000A_000401,臺北北門郵局_臺北901支,25.046920,121.511400,0.573450


In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster



# 以台北車站為中心
center_lat, center_lon = 25.0478, 121.5170
m = folium.Map(location = [center_lat, center_lon], zoom_start = 15)

# 台北車站(圓心)特別標示
folium.CircleMarker(
    location = [center_lat, center_lon],
    radius = 10, # 大小較大
    popup = '台北車站(圓心)',
    color = 'red',
    fill = True,
    fill_color = 'red',
    fill_cpacity = 0.9).add_to(m)
                                           

# 加上其他景點(用MarkerCluster管理)
Attraction_df = pd.read_csv('./attraction_distance_to_Taipei_station.csv', encoding = 'utf-8')

marker_cluster = MarkerCluster().add_to(m)

for idx, row in Attraction_df.iterrows():
    folium.CircleMarker(
    location = [row['PositionLat'], row['PositionLon']],
    radius = 4,
    color = 'blue',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6,
    popup = row['AttractionName']
).add_to(marker_cluster)

m
# m.save('map_with_center_highlight.html')